## Workshop on Scientific Computing in Economics with Python and Julia

## Monday Afternoon Session: Julia for Economists

**Chase Coleman**

## What makes julia the right tool for what we do?

* Fast
* Flexible
* Intuitive
* Quickly growing package ecosystem (We don't want to re-invent the wheel)

## Distributions.jl

A Julia package for probability distributions and associated functions. Particularly, Distributions implements:

* Moments (e.g mean, variance, skewness, and kurtosis), entropy, and other properties
* Probability density/mass functions (pdf) and their logarithm (logpdf)
* Moment generating functions and characteristic functions
* Sampling from population or from a distribution
* Maximum likelihood estimation

In particular, it provides a very natural (and consistent!) way to work with about 70 different distributions.

In [1]:
using Distributions

INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/PDMats.ji for module PDMats.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/Distributions.ji for module Distributions.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/ArrayViews.ji for module ArrayViews.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/StatsFuns.ji for module StatsFuns.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/StatsBase.ji for module StatsBase.


We will create a standard normal distribution and do a variety of things with it below.

In [2]:
# Create the standard normal distribution
nrv = Normal(0.0, 1.0)

# Get the 1st and 99th percentile
nrv_quantiles = quantile(nrv, [0.01, 0.99])

# Get the value of pdf (and cdf) of the 1st and 99th percentile
nrv_pdf = pdf(nrv, nrv_quantiles)
nrv_cdf = cdf(nrv, nrv_quantiles)

# Sample from the distribution (samples 1 draw from distribution unless specified)
rand(nrv)
rand(nrv, 10)

10-element Array{Float64,1}:
 -0.743253
 -0.422246
 -0.073813
 -2.16247 
  0.981967
 -0.491459
  0.25416 
  0.18832 
  0.323087
 -0.613941

In [3]:
typeof(nrv)

Distributions.Normal

Notice that we can use almost exactly the same code -- We only change the distribution name -- to get these same items from the inverse gamma distribution

In [4]:
ivg = InverseGamma(7.0, 4.0)

# Get the 1st and 99th percentile
ivg_quantiles = quantile(ivg, [0.01, 0.99])

# Get the value of pdf (and cdf) of the 1st and 99th percentile
ivg_pdf = pdf(ivg, ivg_quantiles)
ivg_cdf = cdf(ivg, ivg_quantiles)

# Sample from the distribution (samples 1 draw from distribution unless specified)
rand(ivg)
rand(ivg, 10)

10-element Array{Float64,1}:
 0.419918
 0.779849
 0.975664
 1.02611 
 0.430383
 1.50434 
 0.762633
 0.470154
 0.933289
 0.613116

In [5]:
[mean(nrv) mean(ivg)
 std(nrv) std(ivg)
 skewness(nrv) skewness(ivg)
 kurtosis(nrv) kurtosis(ivg)]

4x2 Array{Float64,2}:
 0.0   0.666667
 1.0   0.298142
 0.0   2.23607 
 0.0  12.0     

## Interpolations.jl

This package implements a variety of interpolation schemes for the Julia langauge. It has the goals of ease-of-use, broad algorithmic support, and exceptional performance.

This package is still relatively new. Currently its support is best for B-splines and also supports irregular grids.

In [6]:
using Interpolations

INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/WoodburyMatrices.ji for module WoodburyMatrices.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/AxisAlgorithms.ji for module AxisAlgorithms.


The first thing we need to do is to create an interpolator -- By default this will be defined on [1, Npts]. We need to specify several things:

* Data
* Type of interpolation
  - Order of interpolation
  - Boundary Conditoins
* Where data is located

In [7]:
lb, ub = 0.1, 10.0
x = linspace(lb, ub, 25)
u = log(x)

# Create a BSpline using linear interpolation with data on the grid
lin_itp = interpolate(u, BSpline(Linear()), OnGrid())
qua_itp = interpolate(u, BSpline(Quadratic(Natural())), OnGrid());

We can evaluate the interpolator by using "indexing"

In [8]:
println("Log(x) is: ", round(log(lb), 4))
println("Interpolated value at x is: ", round(lin_itp[lb], 4))

Log(x) is: -2.3026
Interpolated value at x is: -3.7733


What happened??? This is on a grid point, why doesn't it match. This is what I meant when I said it would be defined by default on [1, Npts], we really need to give it the index of the point.

In [9]:
println("Log(x) is: ", round(log(x[1]), 4))
println("Interpolated value at x is: ", round(lin_itp[1], 4))

Log(x) is: -2.3026
Interpolated value at x is: -2.3026


It could get annoying to always be working in indexes for us, but there is a "scaling" feature that allows us to move the domain to where we would like to work.

In [10]:
lin_itp_s = scale(lin_itp, x)
qua_itp_s = scale(qua_itp, x);

In [11]:
println("Log(x) is: ", round(log(lb), 4))
println("Interpolated value at x is: ", round(lin_itp_s[lb], 4))

Log(x) is: -2.3026
Interpolated value at x is: -2.3026


Extrapolation is by default set to linearly extrapolate outside of the bounds, but you can ask it to do

* `Throw` : Throw an error if asked to extrapolate
* `Flat` : Evaluate anything outside grid to the closest grid point
* `Linear` : Linear extrapolate outside of bounds

In [12]:
println(extrapolate(lin_itp_s, Interpolations.Throw())[lb + 0.05])
println(extrapolate(lin_itp_s, Interpolations.Throw())[lb - 0.05])

-2.104508665718352


LoadError: LoadError: BoundsError
while loading In[12], in expression starting on line 2

We can also evaluate the derivatives of our splines

In [13]:
gradient(qua_itp_s, 1.0)

1-element Array{Float64,1}:
 0.950788

Best of all, `Interpolations.jl` is _very_ fast.

In [14]:
n = 10_000_000
fine_grid = linspace(lb, ub, n)

function fill_values(itp::ScaledInterpolation, x::LinSpace)

    # Get number of values and allocate space
    n = length(x)
    interp_values = Array(Float64, n)

    # Fill values
    for i=1:n
        interp_values[i] = itp[x[i]]
    end

    return interp_values
end

@time fill_values(lin_itp_s, fine_grid);

  0.238366 seconds (4.71 k allocations: 76.516 MB, 1.47% gc time)


## NLsolve.jl

In [15]:
using NLsolve

INFO: Precompiling module NLsolve...
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/Distances.ji for module Distances.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/Optim.ji for module Optim.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/Calculus.ji for module Calculus.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/DualNumbers.ji for module DualNumbers.
INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.4/ForwardDiff.ji for module ForwardDiff.


Imagine solving the following system of non-linear equations:


## Optim.jl

For many optimization problems `Optim.jl` will do very well -- However, for more complicated problems, we suggest you look into `NLopt.jl` which is based on the stable (and powerful) `NLopt` C library.

In [16]:
using Optim

Consider the following optimization problem

\begin{align*}
  V(x) &= \max_{c_1, c_2, c_3} v(c_1) + \beta v(c_2) + \beta^2 v(c_3) \\
  \text{subject to } & c_1 + c_2 + c_3 = x
\end{align*}

We will transform this to an unconstrained problem (though this is not necessary) to simplify things. In particular, we will substitute our constraint to get

$$c_3 = x - c_1 - c_2$$

In [72]:
v(c) = -exp(-c)
vp(c) = exp(-c)

function objective(β, x, cvec)
    # Unpack input
    c1, c2 = cvec
    c3 = x - c1 - c2

    # Evaluate objective function
    V = v(c1) + β*v(c2) + β*β*v(c3)

    return V
end

objective (generic function with 1 method)

By default `Optim` minimizes problems; we will minimize the negative of our objective (equivalent to maximizing our original objective)

In [80]:
f(cvec) = -objective(1.0, 1.0, cvec)

res_nd = optimize(f, [0.0, 0.0], NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.33339903759728406,0.3332619921444385]
 * Minimum: 2.149594
 * Iterations: 27
 * Convergence: true
   * |x - x'| < NaN: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
   * |g(x)| < NaN: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 54
 * Gradient Calls: 0

In [81]:
println(Optim.minimizer(res_nd))
println(-Optim.minimum(res_nd))

[0.33339903759728406,0.3332619921444385]
-2.1495939351028284


But we have more information, in this case we could also supply a gradient.

In [82]:
function grad!(β, x, cvec, storage::Vector)
    # Unpack input
    c1, c2 = cvec
    c3 = x - c1 - c2

    # Evaluate objective function
    storage[1] = vp(c1) - β*β*vp(c3)
    storage[2] = β*vp(c2) - β*β*vp(c3)
end

g!(cvec::Vector, storage::Vector) = -grad!(1.0, 1.0, cvec, storage)

g! (generic function with 2 methods)

In [84]:
res_d = optimize(f, [0.0, 0.0], BFGS())

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.3333333333349329,0.3333333333349329]
 * Minimum: 2.149594
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
   * |g(x)| < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 16
 * Gradient Calls: 16

In [132]:
options = OptimizationOptions(ftol=-1.0, grtol=1e-12)
res_d = optimize(f, g!, [0.0, 0.0], method=GradientDescent())

Results of Optimization Algorithm
 * Algorithm: Gradient Descent
 * Starting Point: [0.0,0.0]
 * Minimizer: [-1.872960243837471e-6,-1.872960243837471e-6, ...]
 * Minimum: 2.367882
 * Iterations: 1
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
   * |g(x)| < 1.0e-08: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 74
 * Gradient Calls: 74

In [136]:
fg = DifferentiableFunction(f, g!)
res_d = optimize(fg, [0.0, 0.0], method=NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.33339903759728406,0.3332619921444385]
 * Minimum: 2.149594
 * Iterations: 27
 * Convergence: true
   * |x - x'| < NaN: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-08: true
   * |g(x)| < NaN: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 54
 * Gradient Calls: 0

In [129]:
_x = zeros(2)
g!(Optim.minimizer(res_nd), _x)
_x

2-element Array{Float64,1}:
 -4.30386e-5
  5.5159e-5 

In [130]:
mval = Optim.minimizer(res_nd)
println((f(mval) - f(mval + [0.00001, 0])) / (0.00001))
println((f(mval) - f(mval + [0, 0.00001])) / (0.00001))

-5.020366344865578e-5
4.7993475860153005e-5


## QuantEcon.jl

General purpose tools for 

In [ ]:
using QuantEcon

## Motivating Examples

### Arellano Model of Sovereign Default

[Notebook link](arellano.ipynb)

### Estimating Markov Chain using GMM and SMM

[Notebook link](markov_chain_estimation.ipynb)